In [1]:
from autointent.context.data_handler import Dataset
from autointent.context.utils import load_data

scoring_dataset = load_data("./data/train_data.json")
prediction_dataset = load_data("./data/test_data.json")
len(scoring_dataset.utterances), len(prediction_dataset.utterances)

(165, 57)

In [10]:
from autointent.pipeline.optimization import PipelineOptimizer

config = {
    "nodes": [
        {
            "node_type": "scoring",
            "metric": "scoring_roc_auc",
            "search_space": [
                {"module_type": "knn", "k": [5, 10], "weights": ["uniform", "distance", "closest"], "model_name": ["avsolatorio/GIST-small-Embedding-v0"]},
                {"module_type": "linear", "model_name": ["avsolatorio/GIST-small-Embedding-v0"]},
                # {
                #     "module_type": "dnnc",
                #     "cross_encoder_name": ["cross-encoder/ms-marco-MiniLM-L-6-v2", "avsolatorio/GIST-small-Embedding-v0"],
                #     "search_model_name": ["avsolatorio/GIST-small-Embedding-v0"],
                #     "k": [1, 3],
                #     "train_head": [False, True],
                # },
            ],
        },
        {
            "node_type": "prediction",
            "metric": "prediction_accuracy",
            "search_space": [
                {"module_type": "threshold", "thresh": [0.5]},
                {"module_type": "tunable"},
                # {"module_type": "argmax"},
                # {"module_type": "jinoos"},
            ],
        },
    ]
}

pipeline_optimizer = PipelineOptimizer.from_dict_config(config)

In [ ]:
context = pipeline_optimizer.optimize_from_dataset(scoring_dataset, prediction_dataset) # data with partitions: train_1, train_2, val_1, val_2, test

In [ ]:
context.dump()